'전세사기피해자에게는 금융지원이 제공될 수 있습니다. 국가 및 지방자치단체는 전세사기피해자 및 특정 요건을 충족하는 임차인의 긴박한 주거안정을 보호하기 위해 필요한 자금을 융자하거나 다른 지원을 할 수 있습니다. 또한, 전세사기피해자에게는 주택도시기금에서 주택의 임대차에 필요한 자금을 융자할 수 있는 특례가 존재합니다. 이를 통해 전세사기피해자들의 주거안정을 지원하고 있습니다.'

In [6]:
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1. 문서 내용 읽고 분할 #######################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

# 2. 임베딩 -> 백터 데이터베이스에 저장  #########################

## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(
    model='text-embedding-3-large'
)

## 2.3 벡터 데이터베이스에 생성 및 저장(in local)
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law',
# )

## 2.4 로컬에 저장된 임베딩 데이터 가져오기
database = Chroma(
    collection_name='chroma-law',
    persist_directory='./chroma',
    embedding_function=embedding,
)

In [7]:
# 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색  #############

## 3.1 사용자 질문
query = '전세사기피해자로 인정받기 위한 조건은?'

## 3.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=2)

# 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###########

## 4.1 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 4.2 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)
 
## 4.3 LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI()

## 4.4 LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message.content

'답변: 전세사기피해자로 인정받기 위한 조건은 다음과 같습니다:\n1. 주택의 인도와 주민등록을 마치고 주택임대차계약증서상의 확정일자를 갖추어야 합니다.\n2. 임차인의 임차보증금이 5억원 이하여야 합니다.\n3. 임대인의 파산, 회생절차 개시 또는 임차주택의 경매, 공매절차 개시, 임차인의 집행권원 확보 등에 해당하는 경우에 임차인에게 임차보증금반환채권의 변제를 받지 못하는 피해가 발생하거나 발생할 것이 예상되어야 합니다.\n4. 임대인에 대한 수사 개시, 기망, 임차보증금을 반환할 능력이 없는 자에 대한 임차주택의 양도 또는 임차보증금을 반환할 능력 없이 다수의 주택 취득ㆍ임대 등 임대인이 임차보증금반환채무를 이행하지 않을 의도가 의심되는 상당한 이유가 있어야 합니다.'